#  Creating a Node

The [Hello, FABRIC](./hello_fabric.ipynb) notebook demonstrates how to create a simple node on a random FABRIC site with default characteristics.   Most experiments require more control over the placement and configuration of your nodes.  

This notebook, will show how to set the following properties on a node:

- Site: Choose the FABRIC site where your node will be hosted.
- Host: Choose the specific host that the VM will run on.
- Cores: Number of cores, amount of RAN
- RAM: Amount of RAM
- Disk: Amount of local disk space
- VM Image: The operating system image used for the VM



## Configure the Environment

Set the environment variable that will be used by this notebook. If you are using the FABRIC, JupyterHub some of the environment will be automatically configured for you.  You will only need to set your bastion username, upload your bastion private key, and set the path to where you put your bastion private key. Your bastion username and private key should already be in your possession.  If you do not have a bastion username and private key, please contact the FABRIC admins using the [FABRIC User Forum](https://learn.fabric-testbed.net/forums/) 

If you are using the FABRIC API outside of the JupyterHub you will need to configure all of the environment variables. Defaults below will be correct in many situations but you will need to confirm your configuration.  If you have questions about this configuration, please contact the FABRIC admins using the [FABRIC User Forum](https://learn.fabric-testbed.net/forums/) 

More information about accessing your experiments through the FABRIC bastion hosts can be found [here](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/).

In [ ]:
import os

# If you are using the FABRIC JupyterHub, the following three evnrionment vars
# were automatically provided when you logged in.
#os.environ['FABRIC_CREDMGR_HOST']='cm.fabric-testbed.net'
#os.environ['FABRIC_ORCHESTRATOR_HOST']='orchestrator.fabric-testbed.net'
#os.environ['FABRIC_TOKEN_LOCATION']=os.environ['HOME']+'/work/fabric_token.json'

# Bastion IPs
os.environ['FABRIC_BASTION_HOST'] = 'bastion-1.fabric-testbed.net'

# Set your Bastion username and private key
os.environ['FABRIC_BASTION_USERNAME']=<INSERT_YOUR_FABRIC_USERNAME>
os.environ['FABRIC_BASTION_KEY_LOCATION']=os.environ['HOME']+'/work/fabric_bastion_key'

# Set the keypair FABRIC will install in your slice. 
os.environ['FABRIC_SLICE_PRIVATE_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa'
os.environ['FABRIC_SLICE_PUBLIC_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa.pub'

# If your slice private key uses a passphrase, set the passphrase
#from getpass import getpass
#print('Please input private key passphrase. Press enter for no passphrase.')
#os.environ['FABRIC_SLICE_PRIVATE_KEY_PASSPHRASE']=getpass()

## Setup the Experiment

#### Import the FABRIC API

In [ ]:
import json
import traceback
from fabrictestbed_extensions.fablib.fablib import fablib

In [ ]:
try:
    print(f"{fablib.list_sites()}")
except Exception as e:
    print(f"Exception: {e}")

## Create the Experiment Slice

#### Configure the Experiment Parameters

Choose some specific node attributes.

In [ ]:
slice_name = 'MySlice'
node_name = 'Node1'
site = 'MAX'
host = 'max-w1.fabric-testbed.net'
image = 'default_ubuntu_20'
cores = 2
ram = 8
disk = 10

Create the slice and set the specific node attributes.  Note that the capacities are only *hints*.  The actual values will be the closest instance type to the chosen values.

In [ ]:
try:
    #Create Slice
    slice = fablib.new_slice(slice_name)

    # Add node
    node = slice.add_node(name=node_name, site=site)
    node.set_capacities(cores=cores, ram=ram, disk=disk)
    node.set_image(image)

    #Submit Slice Request
    slice.submit()
    
    slice_id = slice.get_slice_id()
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
try:
    slice = fablib.get_slice(slice_id=slice_id)
    
    print(f"{slice.list_nodes()}")
except Exception as e:
    print(f"Exception: {e}")

#### SSH with fablib

Most experiments will require automated configuration and execution. You can use the fablib library to execute arbitrary commands on your node. 

The following code demonstrates how to use fablib to execute a "Hello, FABRIC" bash script. The library uses the bastion and VM keys defined at the top of this notebook to jump through the bastion host and execute the script.

In [ ]:
command= 'echo Hello, FABRIC from node `hostname -s`'

In [ ]:
try:
    slice = fablib.get_slice(slice_name)
    for node in slice.get_nodes():
        stdout, stderr = node.execute(command)
        print(stdout)
except Exception as e:
    print(f"Fail: {e}")

## Delete Slice

Please delete your slicd when you are done with your experiment.

In [ ]:
try:
    slice = fablib.get_slice(slice_name)
    slice.delete()
except Exception as e:
    print(f"Exception: {e}")